In [ ]:
# importing module
import sys
import os
from pathlib import Path
sys.path.append(str(Path.cwd().parent))
from myutils import *
from SLBQP import SLBQP
from datastorer import DataStorer
import cvxopt
from cvxopt import matrix
from cvxopt import solvers

solvers.options['show_progress'] = False
solvers.options['abstol'] = 1e-12
solvers.options['reltol'] = 1e-12
solvers.options['feastol'] = 1e-12

In [ ]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import time
plt.style.use('seaborn')
params = {
    'figure.figsize': (20, 10),
    'legend.fontsize': 'x-large',
    'axes.labelsize': 'x-large',
    'axes.titlesize':'x-large',
    'xtick.labelsize':'x-large',
    'ytick.labelsize':'x-large',
    'axes.titlesize': 'x-large',
}
plt.rcParams.update(params)

# Parametri default

In [ ]:
seed = 123     # random seed
np.random.seed(seed)
ROSEN = 2
GOLDSTEIN = 1

In [ ]:
epsilons = [10**i for i in range(-2, 1)]
upper_bounds = [0.1, 0.5, 1, 2, 5, 10]
problem_sizes = [100,200,300,400,500,600]
alphas = [0.1, 0.5, 1, 2, 5]
n_prob = 10

In [ ]:
def generate_multiple_alphas(feat, targ, size, alphas, n=n_prob, startdir='./', maxiter=1000):
    """experiments on the alpha paramerer."""
    if not startdir.endswith("/"):
        print("inserting slash in startdir")
        startdir = startdir + "/"
    
    for alpha in alphas:
        alpha_name = str(alpha).replace(".","")
        os.makedirs(f"{startdir}{alpha_name}", exist_ok=True)
        print(f"creating {startdir}{alpha_name}")
    
    for i in range(n):
     
        K, target = sample_transform_problem(feat, targ, size)
        
        sol = solve_with_cvxopt(K, target, 0.1, 1)
        
        with open(f"{startdir}optimal_values.txt", "a") as f:
            f.write(str(sol) + "\n")
            
        for alpha in alphas:
            print(f"Problem {i} {alpha=}")                    
            ds = DataStorer()
            sg, xg, vg, itg = SLBQP(K, target, 1, 0.1, eps=1e-3, maxIter=maxiter, alpha=alpha, prj_type=GOLDSTEIN, ds=ds)          
            alpha_name = str(alpha).replace(".","")
            ds.save_to_file(f"{startdir}{alpha_name}/{i}.csv")
            
            print(f"\t\t{itg=}")
            

In [ ]:
def generate_multiple_epsilons(feat, targ, size, u, epsilons, n, startdir='./'):
    u_name = str(u).replace(".","")
    for e in epsilons:
        eps_name = str(e).replace(".","")
        os.makedirs(f"{startdir}C{u_name}EPS_{eps_name}/{size}/GOLD", exist_ok=True)
        os.makedirs(f"{startdir}C{u_name}EPS_{eps_name}/{size}/ROS", exist_ok=True)

    for i in range(n):
        print(f"generating problem {i}")
        K, target = sample_transform_problem(feat, targ, size)
        for e in epsilons:
            print(f"Solving problem {i} with epsilon {e}")
            eps_name = str(e).replace(".","")
            ds = DataStorer()
            print(f"\tAbout to run GOLDSTEIN with {e=}")
            sg, xg, vg, itg = SLBQP(K, target, u, e, eps=1e-3, maxIter=-1, lmb0=0, d_lmb=2, prj_eps=1e-9, prj_type=GOLDSTEIN, ds=ds)
            ds.save_to_file(f"{startdir}C{u_name}EPS_{eps_name}/{size}/GOLD/{i}.csv")
            ds = DataStorer()
            print(f"\tAbout to run ROSEN with {e=}")
            sg, xg, vg, itg = SLBQP(K, target, u, e, eps=1e-3, maxIter=-1, prj_type=ROSEN, ds=ds)
            ds.save_to_file(f"{startdir}C{u_name}EPS_{eps_name}/{size}/ROS/{i}.csv")
            sol = solve_with_cvxopt(K, target, e, u)
            with open(f"{startdir}C{u_name}EPS_{eps_name}/{size}/optimal_values.txt", "a") as f:
                f.write(str(sol) + "\n")
            
            

In [ ]:
def generate_multiple_upperbounds(feat, targ, size, upper_bounds, epsilon ,n, startdir='./'):
    
    eps_name = str(epsilon).replace(".", "")
    for u in upper_bounds:
        u_name = str(u).replace(".","")
        os.makedirs(f"{startdir}C_{u_name}EPS{eps_name}/{size}/GOLD", exist_ok=True)
        os.makedirs(f"{startdir}C_{u_name}EPS{eps_name}/{size}/ROS", exist_ok=True)
        
    for i in range(n):
        #i = i+10
        print(f"generating problem {i}")
        K, target = sample_transform_problem(feat, targ, size)  
        for u in upper_bounds:
            print(f"Solving problem {i} with U: {u}")
            u_name = str(u).replace(".","")
            ds = DataStorer()
            print(f"\tAbout to run GOLDSTEIN with {u=}")
            sg, xg, vg, itg = SLBQP(K, target, u, epsilon, eps=1e-3, maxIter=-1, lmb0=0, d_lmb=2, prj_eps=1e-9, prj_type=GOLDSTEIN, ds=ds)
            ds.save_to_file(f"{startdir}C_{u_name}EPS{eps_name}/{size}/GOLD/{i}.csv")
            ds = DataStorer()
            print(f"\tAbout to run ROSEN with {u=}")
            sg, xg, vg, itg = SLBQP(K, target, u, epsilon, eps=1e-3, maxIter=-1, prj_type=ROSEN, ds=ds)
            ds.save_to_file(f"{startdir}C_{u_name}EPS{eps_name}/{size}/ROS/{i}.csv")
            
            sol = solve_with_cvxopt(K, target, epsilon, u)
            with open(f"{startdir}C_{u_name}EPS{eps_name}/{size}/optimal_values.txt", "a") as f:
                f.write(str(sol)+"\n")

# ML DATA GENERATION

In [ ]:
feat, targ = load_ml_dataset()
t1 = targ[:,0]
t2 = targ[:,1]
targ = t1
ROOT="./RUNS/ML/"

In [ ]:
for size in problem_sizes:
    generate_multiple_epsilons(feat, targ, size, 1, epsilons, n_prob, startdir=ROOT)

In [ ]:
for size in problem_sizes:
    generate_multiple_upperbounds(feat, targ, size, upper_bounds, 0.1, n_prob, startdir=ROOT)

In [ ]:
ml_alpha_startdir = f"RUNS/ML/ALPHA/"
generate_multiple_alphas(feat, targ, 500, alphas,n_prob, ml_alpha_startdir)

# CALIF DATA GENERATION

In [ ]:
feat, targ = load_california_dataset()
targ
ROOT="./RUNS/CAL/"

In [ ]:
for size in problem_sizes:
    generate_multiple_epsilons(feat, targ, size, 1, epsilons, n_prob, startdir=ROOT)

In [ ]:
for size in problem_sizes:
    generate_multiple_upperbounds(feat, targ, size, upper_bounds, 0.1, n_prob, startdir=ROOT)

In [ ]:
cal_alpha_startdir = f"RUNS/CAL/ALPHA/"
generate_multiple_alphas(feat, targ, 500, alphas,n_prob, cal_alpha_startdir)

# AIRF DATA GENERATION

In [ ]:
feat, targ = load_airfoil_dataset()
targ
ROOT="./RUNS/AIRF/"

In [ ]:
for size in problem_sizes:
    generate_multiple_epsilons(feat, targ, size, 1, epsilons, n_prob, startdir=ROOT)

In [ ]:
for size in problem_sizes:
    generate_multiple_upperbounds(feat, targ, size, upper_bounds, 0.1, n_prob, startdir=ROOT)

In [ ]:
airf_alpha_startdir = f"RUNS/AIRF/ALPHA/"
generate_multiple_alphas(feat, targ, 500, alphas,n_prob, airf_alpha_startdir)